In [2]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import InterpolationMode

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    LearningRateMonitor,
    RichProgressBar,
)
from pytorch_lightning.callbacks.progress.rich_progress import RichProgressBarTheme

import timm

# ─────────── 1) Transforms & Dataset ───────────

def med_transform(img_size=224, gray=True):
    mean, std = ([0.5], [0.5]) if gray else ([0.485,0.456,0.406], [0.229,0.224,0.225])
    return transforms.Compose([
        transforms.Resize((img_size, img_size), antialias=True),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

class ChestNpz(Dataset):
    def __init__(self, npz_path: Path, split: str):
        arr = np.load(npz_path)
        if split == "test":
            self.x, self.y = arr["test_images"], arr["test_labels"]
        else:
            x, y = arr["train_images"], arr["train_labels"]
            n_val = int(0.10 * len(x))
            if split == "train":
                self.x, self.y = x[:-n_val], y[:-n_val]
            else:
                self.x, self.y = x[-n_val:], y[-n_val:]
        self.tf = med_transform(gray=True)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        img = (self.x[idx] * 255).astype("uint8")
        img = Image.fromarray(img).convert("L")
        img = self.tf(img).repeat(3, 1, 1)
        label = torch.from_numpy(self.y[idx]).float()
        return img, label

# ─────────── 2) Self‐Supervised Transforms & DataModule ───────────

class DinoTransform:
    def __init__(self, img_size=224):
        flip   = transforms.RandomHorizontalFlip()
        color  = transforms.RandomApply(
            [transforms.ColorJitter(0.4,0.4,0.2,0.1)], p=0.8
        )
        gray   = transforms.RandomGrayscale(p=0.2)
        normalize = transforms.Normalize([0.5], [0.5])

        self.global_t = transforms.Compose([
            transforms.Resize(img_size, interpolation=InterpolationMode.BICUBIC),
            flip, color, gray,
            transforms.ToTensor(),
            normalize
        ])

    def __call__(self, x):
        g1 = self.global_t(x)
        g2 = self.global_t(x)
        return g1.repeat(3,1,1), g2.repeat(3,1,1)

class ChestNpzSSL(Dataset):
    def __init__(self, npz_path: Path):
        self.arr = np.load(npz_path)["train_images"]
        self.tf  = DinoTransform()

    def __len__(self):
        return len(self.arr)

    def __getitem__(self, idx):
        img = (self.arr[idx] * 255).astype("uint8")
        img = Image.fromarray(img).convert("L")
        return self.tf(img)

class ChestSSLDataModule(pl.LightningDataModule):
    def __init__(self, npz_path, batch=128, workers=0):
        super().__init__()
        self.path = Path(npz_path)
        self.batch = batch
        self.workers = workers

    def setup(self, stage=None):
        self.ds = ChestNpzSSL(self.path)

    def train_dataloader(self):
        return DataLoader(
            self.ds,
            batch_size=self.batch,
            shuffle=True,
            num_workers=self.workers,
            pin_memory=False
        )

# ─────────── 3) Updated DINO Loss with Center Update ───────────

class DinoLoss(nn.Module):
    """
    Simplified DINO loss with center updating to prevent collapse.
    """
    def __init__(self, out_dim=256, teacher_temp=0.04, student_temp=0.1, center_m=0.9):
        super().__init__()
        self.teacher_temp = teacher_temp
        self.student_temp = student_temp
        self.center_m = center_m
        self.register_buffer("center", torch.zeros(1, out_dim))

    def forward(self, student_out, teacher_out):
        # sharpen teacher with current center
        t_out = F.softmax((teacher_out - self.center) / self.teacher_temp, dim=-1)
        s_out = F.log_softmax(student_out / self.student_temp, dim=-1)
        loss = -(t_out * s_out).sum(dim=-1).mean()

        # EMA update of center
        with torch.no_grad():
            batch_center = t_out.mean(dim=0, keepdim=True)
            self.center.mul_(self.center_m).add_(batch_center, alpha=1 - self.center_m)

        return loss

# ─────────── 4) Lightning Module ───────────

class LitDINO(pl.LightningModule):
    def __init__(self, lr=3e-4, out_dim=256, warmup_epochs=5, max_epochs=40):
        super().__init__()
        # student & teacher backbones
        self.student = timm.create_model('vit_tiny_patch16_224', num_classes=0)
        self.teacher = timm.create_model('vit_tiny_patch16_224', num_classes=0)

        embed_dim = self.student.num_features  # 192
        # projection heads
        self.student_head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.GELU(),
            nn.Linear(embed_dim, out_dim),
        )
        self.teacher_head = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.GELU(),
            nn.Linear(embed_dim, out_dim),
        )

        for p in self.teacher.parameters():
            p.requires_grad = False

        self.criterion = DinoLoss(out_dim)
        self.lr = lr
        self.max_epochs = max_epochs
        self.warmup_epochs = warmup_epochs

    def training_step(self, batch, batch_idx):
        g1, g2 = batch
        # student embeddings
        feat1 = self.student(g1)  # [B, 192]
        feat2 = self.student(g2)
        s1 = self.student_head(feat1)
        s2 = self.student_head(feat2)
        # teacher embeddings
        with torch.no_grad():
            t_feat1 = self.teacher(g1)
            t_feat2 = self.teacher(g2)
            t1 = self.teacher_head(t_feat1)
            t2 = self.teacher_head(t_feat2)
        # compute DINO loss
        loss = 0.5 * (self.criterion(s1, t2) + self.criterion(s2, t1))
        self.log("train_loss", loss, prog_bar=True)
        # momentum update teacher
        m = 0.996
        with torch.no_grad():
            for ps, pt in zip(self.student.parameters(), self.teacher.parameters()):
                pt.data.mul_(m).add_(ps.data, alpha=1 - m)
            for hs, ht in zip(self.student_head.parameters(), self.teacher_head.parameters()):
                ht.data.mul_(m).add_(hs.data, alpha=1 - m)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.student.parameters(), lr=self.lr, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.max_epochs)
        return [optimizer], [scheduler]

# ─────────── 5) Progress Callback & Training Entry ───────────

class BatchProgressCallback(pl.callbacks.Callback):
    def on_train_epoch_start(self, *_):
        self.start = time.time()
    def on_train_batch_end(self, trainer, *_):
        if trainer.global_step % 50 == 0:
            mb = torch.cuda.max_memory_allocated() / 1024**2 if torch.cuda.is_available() else 0
            ex_s = (trainer.accumulate_grad_batches * trainer.train_dataloader.batch_size * 50) \
                   / max(1e-3, time.time() - self.start)
            trainer.progress_bar_callback.progress.print(
                f"🖥️ GPU {mb:.0f} MB  ⚡ {ex_s:.1f} img/s"
            )
            self.start = time.time()

if __name__ == "__main__":
    import torch.multiprocessing as mp
    mp.set_start_method("spawn", force=True)

    # DataModule
    npz_path = r"C:\Users\offic\OneDrive\Masaüstü\datasets\SelfSupervised\chestmnist.npz"
    dm_ssl = ChestSSLDataModule(npz_path, batch=128, workers=0)

    # Model
    model_ssl = LitDINO(lr=3e-4, max_epochs=40)

    # Callbacks
    callbacks = [
        ModelCheckpoint(dirpath="checkpoints", save_top_k=-1, every_n_epochs=5),
        LearningRateMonitor(logging_interval='step'),
        RichProgressBar(theme=RichProgressBarTheme(description="blue", progress_bar="green")),
        BatchProgressCallback(),
    ]

    # Trainer
    trainer = Trainer(
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1,
        precision="16-mixed",
        max_epochs=40,
        accumulate_grad_batches=2,
        callbacks=callbacks,
        log_every_n_steps=10,
        enable_progress_bar=True,
    )

    # Start training
    trainer.fit(model_ssl, datamodule=dm_ssl)


Epoch 39/39 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614/614 1:37:57 • 0:00:00 0.14it/s v_num: 7.000 train_loss: 0.002

`Trainer.fit` stopped: `max_epochs=40` reached.
